## Set up the Julep Client

In [1]:
!pip install julep python-dotenv PyYAML --quiet

In [3]:
from dotenv import load_dotenv
import os
from julep import Julep
import yaml


# Load environment variables from the .env file
load_dotenv(override=True)

# Get the julep api key from the environment
api_key = os.getenv("JULEP_API_KEY")

client = Julep(api_key=api_key, environment="production")

# Using fixed UUID to avoid creating new agent/task on each notebook run
AGENT_UUID = "ce7be83e-db8b-4ba9-808e-7cade6812e98"
TASK_UUID = "e623b4dc-9138-440b-a310-b3040764117c"

## Create a Julep Agent

In [ ]:
agent_yaml = yaml.safe_load(open("./agent.yaml"))

agent = client.agents.create_or_update(
    agent_id=AGENT_UUID,
    name=agent_yaml["name"],
    about=agent_yaml["about"],
    instructions=agent_yaml["instructions"],
    model=agent_yaml["model"],
)

## Create a Julep Task

In [48]:
import yaml

# Create the task and attach it to the agent
task = client.tasks.create_or_update(
    agent_id=AGENT_UUID,
    task_id=TASK_UUID,
    **yaml.safe_load(open("./task/full_task.yaml")),
)

## Execute the task

In [ ]:
execution = client.executions.create(
    task_id=TASK_UUID,
    input={
        "url": "https://docs.julep.ai"
    }
)

## Monitor an Execution


In [ ]:
stream = client.executions.status.stream(execution_id=execution.id)

for event in stream:
    print(event.status)
    print("--------------------------------")

In [100]:
import json
transitions = client.executions.transitions.list(execution_id=execution.id).items

for transition in transitions:
    print(transition.type)
    print(transition.current)
    print(json.dumps(transition.output, indent=2))
    print("--------------------------------")

In [ ]:
transition = client.executions.transitions.list(execution_id=execution.id).items[0]

print(transition.output)

## Check the number of documents present in the agent

In [ ]:
# list all the documents in the agent
all_docs = [*client.agents.docs.list(agent_id=AGENT_UUID)]
print(f"Total docs available in the agent: {len(all_docs)}")


for doc in all_docs:
    print(doc.metadata.get("url"))

## Create a Julep Session

In [72]:
session = client.sessions.create(
    agent=AGENT_UUID,
    # Options for RAG search
    recall_options={
        "mode": "hybrid", # Uses both vector and text searches
        "confidence": 0.7, # Confidence threshold for the vector search
        "alpha": 0.5, # Weight of priority of the vector search (0-1)
        "mmr_strength": 0.7, # Maximal Marginal Relevance strength
        "limit": 30, # Number of documents to return
    },
)

## Chat with the agent

In [73]:
def chat_with_agent(query: str) -> str:
    response = client.sessions.chat(
        session_id=session.id,
        messages=[
            {
                "role": "user",
                "content": query
            },
        ],
        recall=True,
        model="claude-3.7-sonnet"
    )
    return response.choices[0].message.content, response.docs

### Start with a question

In [ ]:
agent_response, used_docs = chat_with_agent("How to create a session in julep python sdk?")
print(agent_response)